<header style="width:100%;position:relative">
  <div style="width:80%;float:right;">
    <h1>False Political Claim Detection</h1>
    <h3>Carga y limpieza de los datos - Diccionario y One Hot</h3>
    <h5>Grupo 2</h5>
  </div>
        <img style="width:15%;" src="./images/logo.jpg" alt="UPM" />
</header>

# Índice

1. [Importar Librerias](#1.-Importar-Librerias)  
2. [Carga del Dataset](#2.-Carga-del-Dataset)  
3. [Configuracion de los diccionarios](#3.-Configuracion-de-los-diccionarios)  
4. [Estudio Score](#4.-Estudio-Score)  
5. [Creacion de One-Hot](#5.-Creacion-de-One-Hot)  
6. [One-Hot Variables Categoricas](#6.-One-Hot-Variables-Categoricas)  
7. [Comprobar test](#7.-Comprobar-test)  
8. [Exportar CSV](#8.-Exportar-CSV)  
9. [Referencias](#9-referencias)

# 1. Importar Librerias

In [1]:
import pandas as pd

import json
import spacy

import re

from collections import Counter
from collections import defaultdict

from math import log
from math import sqrt
from math import exp

/home/sergio/anaconda3/envs/anaconda3_r2/lib/python3.9/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


# 2. Carga del Dataset

In [2]:
# Train
url = "formated/train_exportado.csv" 
df_train_exportado = pd.read_csv(url)


# Test
url2 = "formated/test_exportado.csv"
df_test_exportado = pd.read_csv(url2)

Creación de nuevo dataset para guardar solo las columnas que nos interesan

In [3]:
df_train_exportado_words_onehot = df_train_exportado[['id', 'label']]
df_test_exportado_words_onehot = df_test_exportado[['id']]

# 3. Configuracion de los diccionarios

*Statement* selecciada a tratar

In [4]:
statement = 'statement-lemmatize'

Función para limpiar la *statement* y dejar solo las palabras importantes

In [5]:
nlp = spacy.load("en_core_web_lg")  

def clean_statements(statements):
    global nlp
    """
    Elimina pronombres, palabras vacías, palabras irrelevantes y tokens de longitud 1 de las declaraciones.

    Args:
        statements (list of str): Lista de declaraciones a limpiar.
        nlp (spacy.Language): Modelo de spaCy cargado.

    Returns:
        list of str: Lista de declaraciones limpias.
    """
    # Combina las stop words de spaCy con las personalizadas
    custom_stopwords = {"i", "we", "it", "he", "she", "they", "our", "that", "have", "be", "say"}
    all_stopwords = nlp.Defaults.stop_words.union(custom_stopwords)

    cleaned_statements = []
    for statement in statements:
        doc = nlp(statement)
        cleaned_tokens = []
        for token in doc:
            # Excluir palabras que son stop words, están en la lista personalizada o tienen longitud 1
            if token.text.lower() in all_stopwords or token.is_stop or len(token.text) == 1 or len(token.text) == 2:
                continue
            # Incluir solo palabras relevantes
            if token.pos_ in {"VERB", "NOUN", "PROPN", "ADJ"}:
                cleaned_tokens.append(token.text.lower())
        cleaned_statements.append(" ".join(cleaned_tokens))
    return cleaned_statements

Limpiar la *statement* de train y test 

In [6]:
df_train_exportado[statement] = clean_statements(df_train_exportado[statement]) 

In [7]:
df_test_exportado[statement] = clean_statements(df_test_exportado[statement]) 


In [8]:
print(df_train_exportado[statement].head(5))

0        china south china sea military fortress world
1    resources execute months iraq war fund expansi...
2    wisconsin governor propose tax giveaways corpo...
3    representation boyfriend friend family law cas...
4    protests wisconsin propose collective bargaini...
Name: statement-lemmatize, dtype: object


Array de números indicando el número mínimo de veces que aparece en el *dataset*

In [9]:
def generar_array_serie(inicio, fin):
  """
  Genera un array que sigue la serie 4, 7, 11, 16, 22... hasta alcanzar o superar el valor final.

  Args:
    inicio: El primer número de la serie.
    fin: El valor máximo que el último elemento del array no debe superar.

  Returns:
    Una lista (array) con los números de la serie.
  """
  array = [inicio]
  ultimo_elemento = inicio
#   incremento = 3
  incremento = 1
  while ultimo_elemento < fin:
    ultimo_elemento += incremento
    array.append(ultimo_elemento)
    incremento += 1
  return array

*document_frequency*:
Esta función calcula la frecuencia de documentos en los que aparece cada palabra única dentro de una lista de documentos tokenizados.

*filtra*:
Filtra palabras de un diccionario según una frecuencia mínima y una proporción máxima de aparición en los documentos, devolviendo solo las palabras que cumplen ambos criterios.

In [10]:
# Cuenta en cuántos documentos aparece cada palabra
def document_frequency(tokenized_docs):
    df_counter = Counter()
    for doc in tokenized_docs:
        unique_words = set(doc)
        df_counter.update(unique_words)
    return df_counter

# Función para filtrar palabras por frecuencia mínima y máxima proporción
def filtra(dic_frecuencia, doc_frecuencia, total_docs, min_freq, max_prop):
    return {
        word: dic_frecuencia[word]  # frecuencia total (recuento)
        for word in dic_frecuencia
        if doc_frecuencia[word] >= min_freq and (doc_frecuencia[word] / total_docs) <= max_prop
    }

Selección de parametros para aplicar seleccionar la frecuencia mínima de aparición y la probabilidad máxima de aparición para eliminar palabras que aparecen en demasiadas *statements*

In [11]:
# Lista de valores mínimos de frecuencia. Una palabra debe aparecer al menos este número de veces en los documentos para ser incluida en el diccionario.
# min_freq_values = [5, 10, 20, 50]
# min_freq_values = [4,7,10 ,20, 35, 55 ,80,105, 135]
min_freq_values =  generar_array_serie(4, 270)
# mi_array = generar_array_serie(2, 270)

# Lista de proporciones máximas. Una palabra no debe aparecer en más del porcentaje especificado de documentos para ser incluida en el diccionario.
# max_prop_values = [0.5, 0.65, 0.8]
max_prop_values = [0.4]


Dividir el *datset* entre los que tienen *label* igual a 0 o 1

In [12]:
# Tokenización según clase
statements_true = [str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 1]
statements_false = [str(text).split() for text, label in zip(df_train_exportado[statement], df_train_exportado['label']) if label == 0]

print(len(statements_true))
print(len(statements_false))


5795
3155


Sacar la proporción entre la cantidad de filas con *label* igual a 0 o a 1

In [13]:
n_docs_true = len(statements_true)
n_docs_false = len(statements_false)
adjust_ratio = n_docs_true / n_docs_false

Se cuentan las palabras únicas en los documentos clasificados como verdaderos  y falsos, y se calcula la frecuencia de documentos en los que aparece cada palabra.

In [14]:
# Contadores
counter_true = Counter(word for doc in statements_true for word in doc)
counter_false = Counter(word for doc in statements_false for word in doc)
df_true = document_frequency(statements_true)
df_false = document_frequency(statements_false)

Creación de los diferentes diccionarios de frecuencia de las palabras

In [15]:
# Procesa combinaciones
for min_freq_true in min_freq_values:
    min_freq_false = max(1, round(min_freq_true / adjust_ratio))  # Ajuste proporcional

    for max_prop in max_prop_values:
        # Filtrado
        filtered_true = filtra(counter_true, df_true, n_docs_true, min_freq_true, max_prop)
        filtered_false = filtra(counter_false, df_false, n_docs_false, min_freq_false, max_prop)

        # Palabras en común
        common_words = set(filtered_true.keys()) & set(filtered_false.keys())

        # Exclusivos (frecuencia absoluta)
        exclusive_true = {
            word: freq for word, freq in filtered_true.items() if word not in common_words
        }
        exclusive_false = {
            word: freq for word, freq in filtered_false.items() if word not in common_words
        }

        # Opcional: ordenar por frecuencia descendente
        filtered_true = dict(sorted(filtered_true.items(), key=lambda x: x[1], reverse=True))
        filtered_false = dict(sorted(filtered_false.items(), key=lambda x: x[1], reverse=True))
        exclusive_true = dict(sorted(exclusive_true.items(), key=lambda x: x[1], reverse=True))
        exclusive_false = dict(sorted(exclusive_false.items(), key=lambda x: x[1], reverse=True))

        # Guarda resultados
        base = f"nb{min_freq_true}_na{int(max_prop * 100)}"
        # with open(f"json/dictionaries/true_{base}.json", "w") as f:
        #     json.dump(filtered_true, f, indent=2)
        # with open(f"json/dictionaries/false_{base}.json", "w") as f:
        #     json.dump(filtered_false, f, indent=2)
        with open(f"json/dictionaries/true_exclusive_{base}.json", "w") as f:
            json.dump(exclusive_true, f, indent=2)
        with open(f"json/dictionaries/false_exclusive_{base}.json", "w") as f:
            json.dump(exclusive_false, f, indent=2)

        print(f"✔ Guardado con min_freq_true={min_freq_true}, min_freq_false={min_freq_false}, max_prop={max_prop}")


✔ Guardado con min_freq_true=4, min_freq_false=2, max_prop=0.4
✔ Guardado con min_freq_true=5, min_freq_false=3, max_prop=0.4
✔ Guardado con min_freq_true=7, min_freq_false=4, max_prop=0.4
✔ Guardado con min_freq_true=10, min_freq_false=5, max_prop=0.4
✔ Guardado con min_freq_true=14, min_freq_false=8, max_prop=0.4
✔ Guardado con min_freq_true=19, min_freq_false=10, max_prop=0.4
✔ Guardado con min_freq_true=25, min_freq_false=14, max_prop=0.4
✔ Guardado con min_freq_true=32, min_freq_false=17, max_prop=0.4
✔ Guardado con min_freq_true=40, min_freq_false=22, max_prop=0.4
✔ Guardado con min_freq_true=49, min_freq_false=27, max_prop=0.4
✔ Guardado con min_freq_true=59, min_freq_false=32, max_prop=0.4
✔ Guardado con min_freq_true=70, min_freq_false=38, max_prop=0.4
✔ Guardado con min_freq_true=82, min_freq_false=45, max_prop=0.4
✔ Guardado con min_freq_true=95, min_freq_false=52, max_prop=0.4
✔ Guardado con min_freq_true=109, min_freq_false=59, max_prop=0.4
✔ Guardado con min_freq_true=124

Funcíón para calcular el peso de los diferentes diccionarios según su frecuencia de aparición

In [16]:
diccionario_true = []
diccionario_false = []

def calculate_weights(nb):
    """
    Calcula los pesos para los diccionarios true y false basados en los valores de nb.

    Args:
        nb (list): Lista de valores mínimos de frecuencia.

    Returns:
        tuple: Dos listas de tuplas con rutas de diccionarios y sus pesos correspondientes.
    """
    diccionario_true = []
    diccionario_false = []

    for n in nb:

    # Selección del metodo de evaluación de pesosa
        ## Logaritmos normalizados
        # weight = 1 - log(n) / log(max(nb))
        
        ## Raiz de logaritmos
        # weight = 1 / sqrt(log(n))

        ## Exponencial negativa:
        # weight = exp(-n / max(nb))
        
        ## Inversa al cuadrado:
        # weight = 1 / (n ** 2)

        ## Peso por rareza relativa: 67,9
        weight = 1 / (n - min(nb) + 1)

        ## Escalado lineal invertido:
        # weight = (max(nb) - n) / (max(nb) - min(nb))

        diccionario_true.append((f"json/dictionaries/true_nb{n}_na40.json", weight))
        diccionario_false.append((f"json/dictionaries/false_nb{n}_na40.json", weight))
    
    return diccionario_true, diccionario_false

Función para crear el diccionario ponderado en el que se indica el valor de cada palabra

In [17]:
def construir_diccionario_ponderado(lista_archivos_y_pesos):
    resultado = defaultdict(float)
    for ruta, peso in lista_archivos_y_pesos:
        with open(ruta, "r") as f:
            datos = json.load(f)
            for palabra in datos:
                resultado[palabra] += peso
    return dict(resultado)

Calcular el peso de de las palabras en los diccioanrios de true o de false

In [18]:
# Llamar a la función
diccionario_true, diccionario_false = calculate_weights(min_freq_values)

Creación de los diccionarios ponderadaos finales

In [19]:
# Crear diccionarios ponderados
diccionario_true_ponderado = construir_diccionario_ponderado(diccionario_true)
diccionario_false_ponderado = construir_diccionario_ponderado(diccionario_false)

# Opcional: guardar resultados
with open("json/dictionaries_ponderados/diccionario_true_ponderado.json", "w") as f:
    json.dump(diccionario_true_ponderado, f, indent=2)

with open("json/dictionaries_ponderados/diccionario_false_ponderado.json", "w") as f:
    json.dump(diccionario_false_ponderado, f, indent=2)


Función para calcular el *score* de las diferentes *statement* en función de los diccionarios ponderados

In [20]:
# Función para calcular el valor de una statement
def calcular_score(statement):
    # Preprocesar: pasar a minúsculas, quitar puntuación
    palabras = re.findall(r"\b\w+\b", statement.lower())
    score = 0.0
    for palabra in palabras:
        score += diccionario_true_ponderado.get(palabra, 0)
        score -= diccionario_false_ponderado.get(palabra, 0)
    return score


Crear los *scores* de los diferentes *statement* del *dataset* 

In [21]:
# Cargar los diccionarios ponderados
with open("json/dictionaries_ponderados/diccionario_true_ponderado.json") as f:
    diccionario_true_ponderado = json.load(f)

with open("json/dictionaries_ponderados/diccionario_false_ponderado.json") as f:
    diccionario_false_ponderado = json.load(f)

# Aplicar la función al dataset
df_train_exportado_words_onehot["statement_score"] = df_train_exportado[statement].apply(calcular_score)
df_test_exportado_words_onehot["statement_score"] = df_test_exportado[statement].apply(calcular_score)

print(df_train_exportado_words_onehot.head(5))
# Guardar el dataset con la nueva columna (opcional)
# df.to_csv("data/train_con_score.csv", index=False)

            id  label  statement_score
0  81f884c64a7      1         1.313367
1  30c2723a188      0        -2.288087
2  6936b216e5d      0        -0.996092
3  b5cd9195738      1        -2.321938
4  84f8dac7737      0        -3.824482


/tmp/ipykernel_488177/2552788845.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_exportado_words_onehot["statement_score"] = df_train_exportado[statement].apply(calcular_score)
/tmp/ipykernel_488177/2552788845.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_exportado_words_onehot["statement_score"] = df_test_exportado[statement].apply(calcular_score)


Calcular el porentaje de aciertos en *train* según los *scores*

# 4 Estudio score


# 5. Creacion de One-Hot

Crear One-Hot de las 60 palabrás más importantes 

# 6. One-Hot Variables Categoricas 

Creación de los One-Hot de las columnas categóricas

## 6.1 Subject

In [22]:
# print(df_train_exportado.columns)

# Filtrar las columnas que comienzan con 'subject_'
subject_columns = [col for col in df_train_exportado.columns if col.startswith('subject-')]
print(subject_columns)
# Agregar las columnas filtradas al DataFrame words_onehot
df_train_exportado_words_onehot = pd.concat([df_train_exportado_words_onehot, df_train_exportado[subject_columns]], axis=1)
df_test_exportado_words_onehot = pd.concat([df_test_exportado_words_onehot, df_test_exportado[subject_columns]], axis=1)

print("Columnas 'subject_' añadidas correctamente a los DataFrames words_onehot.")

['subject-ciencia_y_tecnologia', 'subject-economia_y_finanzas', 'subject-educacion', 'subject-medio_ambiente_y_energia', 'subject-otros', 'subject-politica_y_gobierno', 'subject-relaciones_internacionales', 'subject-salud_y_bienestar', 'subject-seguridad_y_justicia', 'subject-sociedad_y_cultura', 'subject-trabajo_y_empleo']
Columnas 'subject_' añadidas correctamente a los DataFrames words_onehot.


## 6.2 speaker

In [23]:
df_train_exportado_words_onehot['speaker_grouped'] = df_train_exportado['speaker_grouped']
df_test_exportado_words_onehot['speaker_grouped'] = df_test_exportado['speaker_grouped']

In [24]:
df_train_exportado_words_onehot = pd.get_dummies(df_train_exportado_words_onehot, columns=['speaker_grouped'], prefix='speaker_grouped', dtype=int)
df_test_exportado_words_onehot = pd.get_dummies(df_test_exportado_words_onehot, columns=['speaker_grouped'], prefix='speaker_grouped', dtype=int)

## 6.3 speaker_job

In [25]:
df_train_exportado_words_onehot['speaker_job_grouped'] = df_train_exportado['speaker_job_grouped']
df_test_exportado_words_onehot['speaker_job_grouped'] = df_test_exportado['speaker_job_grouped']

In [26]:
df_train_exportado_words_onehot = pd.get_dummies(df_train_exportado_words_onehot, columns=['speaker_job_grouped'], prefix='speaker_job_grouped', dtype=int)
df_test_exportado_words_onehot = pd.get_dummies(df_test_exportado_words_onehot, columns=['speaker_job_grouped'], prefix='speaker_job_grouped', dtype=int)

## 6.4 state_info

In [27]:
df_test_exportado_words_onehot['state_info_grouped'] = df_test_exportado['state_info_grouped']
df_train_exportado_words_onehot['state_info_grouped'] = df_train_exportado['state_info_grouped']

In [28]:
# dfc = pd.get_dummies(dfc, columns=['state_info_grouped'], prefix='state_info_grouped', dtype=int)
df_train_exportado_words_onehot = pd.get_dummies(df_train_exportado_words_onehot, columns=['state_info_grouped'], prefix='state_info_grouped', dtype=int)
df_test_exportado_words_onehot = pd.get_dummies(df_test_exportado_words_onehot, columns=['state_info_grouped'], prefix='state_info_grouped', dtype=int)

## 6.5 party_affilation

In [29]:
df_test_exportado_words_onehot['party_group'] = df_test_exportado['party_group']
df_train_exportado_words_onehot['party_group'] = df_train_exportado['party_group']

In [30]:
# dfc = pd.get_dummies(dfc, columns=['party_group'], prefix='party_group', dtype=int)
df_train_exportado_words_onehot = pd.get_dummies(df_train_exportado_words_onehot, columns=['party_group'], prefix='party_group', dtype=int)
df_test_exportado_words_onehot = pd.get_dummies(df_test_exportado_words_onehot, columns=['party_group'], prefix='party_group', dtype=int)

# 7. Comprobar test

In [31]:
df_test_exportado_words_onehot.columns

Index(['id', 'statement_score', 'subject-ciencia_y_tecnologia',
       'subject-economia_y_finanzas', 'subject-educacion',
       'subject-medio_ambiente_y_energia', 'subject-otros',
       'subject-politica_y_gobierno', 'subject-relaciones_internacionales',
       'subject-salud_y_bienestar', 'subject-seguridad_y_justicia',
       'subject-sociedad_y_cultura', 'subject-trabajo_y_empleo',
       'speaker_grouped_barack-obama', 'speaker_grouped_chain-email',
       'speaker_grouped_donald-trump', 'speaker_grouped_hillary-clinton',
       'speaker_grouped_john-mccain', 'speaker_grouped_marco-rubio',
       'speaker_grouped_mitt-romney', 'speaker_grouped_other_speakers',
       'speaker_grouped_rick-perry', 'speaker_grouped_rick-scott',
       'speaker_grouped_scott-walker',
       'speaker_job_grouped_business_professional',
       'speaker_job_grouped_federal_legislative_candidate',
       'speaker_job_grouped_federal_legislative_leader',
       'speaker_job_grouped_federal_legislator_h

Contador de columnas que más aparecen

In [ ]:
import json

# Seleccionar las columnas de tipo One-Hot (excluyendo 'id' y 'statement_score')
one_hot_columns = [col for col in df_test_exportado_words_onehot.columns if col not in ['id', 'statement_score']]

# Contar la cantidad de unos en cada columna
one_hot_counts = df_test_exportado_words_onehot[one_hot_columns].sum().sort_values()

# Convertir la salida a un diccionario
one_hot_counts_dict = one_hot_counts.to_dict()

# Exportar a un archivo JSON
with open('json/one_hot_counts.json', 'w') as json_file:
    json.dump(one_hot_counts_dict, json_file, indent=4)

print("La salida ha sido exportada a 'one_hot_counts.json'.")

La salida ha sido exportada a 'one_hot_counts.json'.


# 8. Exportar CSV

In [33]:
# Comparar las columnas de dfc y dfa
columns_train = set(df_train_exportado_words_onehot.columns)
columns_test = set(df_test_exportado_words_onehot.columns)

# Encontrar las diferencias
differences = columns_train.symmetric_difference(columns_test)

# Verificar si la única diferencia es 'id' y 'label'
if differences == {'id', 'label'}:
    print("La única diferencia entre las columnas es 'id' y 'label'.")
else:
    print("Existen otras diferencias en las columnas:", differences)
    print("Columnas en train pero no en test:", columns_train - columns_test)
    print("Columnas en test pero no en train:", columns_test - columns_train)

Existen otras diferencias en las columnas: {'label'}
Columnas en train pero no en test: {'label'}
Columnas en test pero no en train: set()


In [34]:
# Exportar los datasets combinados a archivos CSV
df_train_exportado_words_onehot.to_csv('./formated/train_exportado_words.csv', index=False)
df_test_exportado_words_onehot.to_csv('./formated/test_exportado_words.csv', index=False)

print("Datasets combinados y exportados correctamente.")

Datasets combinados y exportados correctamente.


# 9. Referencias

* [pandas documentation — pandas 2.2.3 documentation. (s. f.).](https://pandas.pydata.org/docs/)  